# Cystisoma corneas

## Inference

First open your heatmap in napari. 

In [6]:
# import modules
import torchio as tio

# load in the nifti file with torchio
# 2nd scan
# image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii'

# Poor scans (not used)
#image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_1448930_head/Cystisoma_1448930_head0.nii'
#image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190212_01_head/FEG20190212_010.nii'
# image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_sp_FEG221129_366/FEG221129_366.nii'
# image = tio.ScalarImage(image_path)



In [7]:
image

ScalarImage(shape: (1, 1446, 1657, 1964); spacing: (1.00, 1.00, 1.00); orientation: LPS+; path: "/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii")

In [8]:
# view file with napari
import napari


viewer = napari.view_image(image.numpy(), name='image')


/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped


In napari, label a few of the features (that are next to each other) to estimate the resample ratio, then save it as a csv file and load it in the next step


In [18]:
import pandas as pd
# 2nd scan
#RR = pd.read_csv('/home/jake/projects/dhr/FEG20190213_01_corneas_resample_ratio.csv')
# 3rd scan
RR = pd.read_csv('/home/jake/projects/dhr/Cystisoma_sp_FEG221129_366_corneas_estimating_resample_ratio.csv')

RR

,index,axis-0,axis-1,axis-2,axis-3
0,0.0,0.0,643.0,780.704145,299.347616
1,1.0,0.0,643.0,793.346004,317.046218
2,2.0,0.0,643.0,808.516233,336.009005
3,3.0,0.0,643.0,828.111113,354.339699
4,4.0,0.0,643.0,847.073901,374.566672
5,5.0,0.0,643.0,866.668781,389.736902
6,6.0,0.0,643.0,919.132492,394.793645
7,7.0,0.0,643.0,893.216683,380.887601
8,8.0,0.0,643.0,871.093431,363.189000
9,9.0,0.0,643.0,852.130644,345.490398


In [19]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-1', 'axis-2', 'axis-3']].values
dist_matrix = distance_matrix(coordinates, coordinates) #Create a distance matrix


In [20]:
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the dataframe
RR['nearest_distance'] = nearest_distances
RR

,index,axis-0,axis-1,axis-2,axis-3,nearest_distance
0,0.0,0.0,643.0,780.704145,299.347616,21.749875
1,1.0,0.0,643.0,793.346004,317.046218,21.749875
2,2.0,0.0,643.0,808.516233,336.009005,24.284216
3,3.0,0.0,643.0,828.111113,354.339699,25.597812
4,4.0,0.0,643.0,847.073901,374.566672,24.780944
5,5.0,0.0,643.0,866.668781,389.736902,24.780944
6,6.0,0.0,643.0,919.132492,394.793645,29.411005
7,7.0,0.0,643.0,893.216683,380.887601,27.983946
8,8.0,0.0,643.0,871.093431,363.189000,25.938924
9,9.0,0.0,643.0,852.130644,345.490398,25.480480


In [21]:
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance

23.151474885249055

In [22]:
resample_ratio = mean_nearest_distance/10 #Makes average distance between features 10 voxels
resample_ratio

2.3151474885249055

## Inference

In [1]:
!python main.py infer -h #If you need help for parameters in infer

usage: main.py [-h] [--volume_path VOLUME_PATH]
               [--sql_storage_url SQL_STORAGE_URL]
               [--starting_weights_path STARTING_WEIGHTS_PATH]
               [--model_path MODEL_PATH] [--profile] [--num_steps NUM_STEPS]
               [--check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH]
               [--resample_ratio_path RESAMPLE_RATIO_PATH]
               [--bbox_path BBOX_PATH] [--n_x_dirs N_X_DIRS]
               [--n_y_dirs N_Y_DIRS] [--n_z_dirs N_Z_DIRS]
               [--resample_ratio RESAMPLE_RATIO] [--already_resampled]
               [--training_data_histogram] [--infer_with_max_filter_fun]
               [--average_threshold AVERAGE_THRESHOLD]
               [--inference_peak_min_val INFERENCE_PEAK_MIN_VAL]
               {train,tune,infer,locate_peaks} config_path

Train, hyperparameter tune or infer with a deep heatmap regression model

positional arguments:
  {train,tune,infer,locate_peaks}
                        Mode of operation
  config_path      

Make inference first with 1 direction in the x, y, and z axis before creating inference in 3 directions. This will produce three files: two csv files (one for the resampled space, and the other not), and a nifty file with predictions.

*Note: these files are currently saved to /home/jake/projects/dhr/output/

In [1]:
#Inference with 1 direction in x, y, and z for 2nd scan
#!python main.py infer configs/cystisoma_corneas.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m .logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 1 -ny 1 -nz 1

#3rd scan
!python main.py infer configs/cystisoma_corneas.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_sp_FEG221129_366/FEG221129_366.nii' -m ./logs/cystisoma_corneas/lightning_logs/version_14/ -rr 2.3151474885249055 -nx 1 -ny 1 -nz 1

TorchIO version: 0.18.88
Running setup!
Found 2 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 2 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   2
Validation:  2
Test:       2
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_length': 9, 'batch_size': 2, "
 "'config_stem': 'cystisoma_corneas', 'correct_prediction_distance': 6, "
 "'debug_plots': False, 'double_channels_with_depth': True, 'dropout': "
 "0.0175756794552526, 'heatmap_max_length': 64, 'histogra

You can then try in more directions (3 direction in x, y, z is usually best) and modify parameters such as max filter (-mxf) (although this usually doesn't work well with Cystisoma), average threshold (-at), and peak minimum values (-ipmv) need be.

In [ ]:
#Attempts of inferences for 2nd scan (best model should be found on last commented line in this kernel)
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 1
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 1.5
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 2
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.5
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.5 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.01 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.02
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.03
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.04 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.05
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.1
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.2 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.3 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.4
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.02 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.03 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.03 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.03 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.03 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.03 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.03 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.01 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.01 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.01 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.01 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.01 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.01
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.02
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.4 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.3 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.2 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.1 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.05 --infer_with_max_filter_fun


#Best inference for 2nd scan
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5 

In [3]:
#Scan 3 (not used)
# !python main.py infer configs/cystisoma_corneas.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_sp_FEG221129_366/FEG221129_366.nii' -m ./logs/cystisoma_corneas/lightning_logs/version_14/ -rr 2.3151474885249055 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5

TorchIO version: 0.18.88
Running setup!
Found 2 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 2 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   2
Validation:  2
Test:       2
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_length': 9, 'batch_size': 2, "
 "'config_stem': 'cystisoma_corneas', 'correct_prediction_distance': 6, "
 "'debug_plots': False, 'double_channels_with_depth': True, 'dropout': "
 "0.0175756794552526, 'heatmap_max_length': 64, 'histogra

 37%|███████████████▊                           | 36/98 [00:18<00:31,  1.98it/s]


 38%|████████████████▏                          | 37/98 [00:19<00:30,  1.98it/s]


 39%|████████████████▋                          | 38/98 [00:19<00:30,  1.94it/s]


 40%|█████████████████                          | 39/98 [00:20<00:30,  1.94it/s]


 41%|█████████████████▌                         | 40/98 [00:20<00:29,  1.94it/s]


 42%|█████████████████▉                         | 41/98 [00:21<00:29,  1.96it/s]


 43%|██████████████████▍                        | 42/98 [00:21<00:28,  1.98it/s]


 44%|██████████████████▊                        | 43/98 [00:22<00:27,  1.99it/s]


 45%|███████████████████▎                       | 44/98 [00:22<00:27,  1.99it/s]


 46%|███████████████████▋                       | 45/98 [00:23<00:26,  2.00it/s]


 47%|████████████████████▏                      | 46/98 [00:23<00:25,  2.00it/s]


 48%|████████████████████▌                      | 47/98 [00:24<00:25,  2.00it/s]


 49%

  4%|█▊                                          | 4/98 [00:02<00:48,  1.93it/s]


  5%|██▏                                         | 5/98 [00:02<00:47,  1.95it/s]


  6%|██▋                                         | 6/98 [00:03<00:46,  1.98it/s]


  7%|███▏                                        | 7/98 [00:03<00:45,  1.99it/s]


  8%|███▌                                        | 8/98 [00:04<00:45,  2.00it/s]


  9%|████                                        | 9/98 [00:04<00:44,  2.00it/s]


 10%|████▍                                      | 10/98 [00:05<00:43,  2.00it/s]


 11%|████▊                                      | 11/98 [00:05<00:43,  2.00it/s]


 12%|█████▎                                     | 12/98 [00:06<00:42,  2.01it/s]


 13%|█████▋                                     | 13/98 [00:06<00:42,  2.01it/s]


 14%|██████▏                                    | 14/98 [00:07<00:41,  2.01it/s]


 15%|██████▌                                    | 15/98 [00:07<00:41,  2.01it/s]


 16%

 80%|██████████████████████████████████▏        | 78/98 [00:39<00:10,  1.98it/s]


 81%|██████████████████████████████████▋        | 79/98 [00:39<00:09,  1.97it/s]


 82%|███████████████████████████████████        | 80/98 [00:40<00:09,  1.97it/s]


 83%|███████████████████████████████████▌       | 81/98 [00:40<00:08,  1.97it/s]


 84%|███████████████████████████████████▉       | 82/98 [00:41<00:08,  1.98it/s]


 85%|████████████████████████████████████▍      | 83/98 [00:41<00:07,  1.97it/s]


 86%|████████████████████████████████████▊      | 84/98 [00:42<00:07,  1.97it/s]


 87%|█████████████████████████████████████▎     | 85/98 [00:42<00:06,  1.98it/s]


 88%|█████████████████████████████████████▋     | 86/98 [00:43<00:06,  1.97it/s]


 89%|██████████████████████████████████████▏    | 87/98 [00:43<00:05,  1.97it/s]


 90%|██████████████████████████████████████▌    | 88/98 [00:44<00:05,  1.98it/s]


 91%|███████████████████████████████████████    | 89/98 [00:44<00:04,  1.98it/s]


 92%

 47%|████████████████████▏                      | 46/98 [00:23<00:26,  1.99it/s]


 48%|████████████████████▌                      | 47/98 [00:23<00:25,  1.99it/s]


 49%|█████████████████████                      | 48/98 [00:24<00:25,  1.99it/s]


 50%|█████████████████████▌                     | 49/98 [00:24<00:24,  1.99it/s]


 51%|█████████████████████▉                     | 50/98 [00:25<00:24,  1.99it/s]


 52%|██████████████████████▍                    | 51/98 [00:25<00:23,  1.99it/s]


 53%|██████████████████████▊                    | 52/98 [00:26<00:23,  1.99it/s]


 54%|███████████████████████▎                   | 53/98 [00:26<00:22,  1.99it/s]


 55%|███████████████████████▋                   | 54/98 [00:27<00:22,  2.00it/s]


 56%|████████████████████████▏                  | 55/98 [00:27<00:21,  1.99it/s]


 57%|████████████████████████▌                  | 56/98 [00:28<00:21,  1.98it/s]


 58%|█████████████████████████                  | 57/98 [00:28<00:21,  1.94it/s]


 59%

...

  9%|████                                        | 9/98 [00:04<00:44,  2.00it/s]


 10%|████▍                                      | 10/98 [00:05<00:43,  2.00it/s]


 11%|████▊                                      | 11/98 [00:05<00:43,  2.01it/s]


 12%|█████▎                                     | 12/98 [00:06<00:42,  2.01it/s]


 13%|█████▋                                     | 13/98 [00:06<00:42,  2.01it/s]


 14%|██████▏                                    | 14/98 [00:07<00:41,  2.01it/s]


 15%|██████▌                                    | 15/98 [00:07<00:41,  2.01it/s]


 16%|███████                                    | 16/98 [00:08<00:40,  2.01it/s]


 17%|███████▍                                   | 17/98 [00:08<00:40,  2.01it/s]


 18%|███████▉                                   | 18/98 [00:09<00:39,  2.01it/s]


 19%|████████▎                                  | 19/98 [00:09<00:39,  2.01it/s]


 20%|████████▊                                  | 20/98 [00:10<00:38,  2.01it/s]


 21%

Rotating volume back to original orientation...


100%|█████████████████████████████████████████████| 3/3 [03:54<00:00, 78.31s/it]

                                                                                
 33%|██████████████▋                             | 1/3 [03:54<07:49, 234.50s/it]

  0%|                                                     | 0/3 [00:00<?, ?it/s]
Running inference with 120° rotation along the x-axis (3/3),
60° rotation along the y-axis (2/3)
and 0° rotation along the z-axis (1/3)
Creating sampler and applying transform to image...
Initialising patch_loader and aggregator...
Starting inference...



  0%|                                                    | 0/98 [00:00<?, ?it/s]


  1%|▍                                           | 1/98 [00:00<01:02,  1.54it/s]


  2%|▉                                           | 2/98 [00:01<00:53,  1.78it/s]


  3%|█▎                                          | 3/98 [00:01<00:50,  1.88it/s]


  4%|█▊                            

 67%|████████████████████████████▉              | 66/98 [00:33<00:16,  1.98it/s]


 68%|█████████████████████████████▍             | 67/98 [00:33<00:15,  1.98it/s]


 69%|█████████████████████████████▊             | 68/98 [00:34<00:15,  1.99it/s]


 70%|██████████████████████████████▎            | 69/98 [00:34<00:14,  1.99it/s]


 71%|██████████████████████████████▋            | 70/98 [00:35<00:14,  1.99it/s]


 72%|███████████████████████████████▏           | 71/98 [00:35<00:13,  2.00it/s]


 73%|███████████████████████████████▌           | 72/98 [00:36<00:13,  1.99it/s]


 74%|████████████████████████████████           | 73/98 [00:36<00:12,  2.00it/s]


 76%|████████████████████████████████▍          | 74/98 [00:37<00:12,  2.00it/s]


 77%|████████████████████████████████▉          | 75/98 [00:37<00:11,  1.99it/s]


 78%|█████████████████████████████████▎         | 76/98 [00:38<00:11,  1.99it/s]


 79%|█████████████████████████████████▊         | 77/98 [00:38<00:10,  1.99it/s]


 80%

 34%|██████████████▍                            | 33/98 [00:16<00:32,  2.00it/s]


 35%|██████████████▉                            | 34/98 [00:17<00:32,  2.00it/s]


 36%|███████████████▎                           | 35/98 [00:17<00:31,  2.00it/s]


 37%|███████████████▊                           | 36/98 [00:18<00:30,  2.00it/s]


 38%|████████████████▏                          | 37/98 [00:18<00:30,  2.00it/s]


 39%|████████████████▋                          | 38/98 [00:19<00:29,  2.01it/s]


 40%|█████████████████                          | 39/98 [00:19<00:29,  2.01it/s]


 41%|█████████████████▌                         | 40/98 [00:20<00:28,  2.01it/s]


 42%|█████████████████▉                         | 41/98 [00:20<00:28,  2.01it/s]


 43%|██████████████████▍                        | 42/98 [00:21<00:28,  1.99it/s]


 44%|██████████████████▊                        | 43/98 [00:21<00:27,  2.00it/s]


 45%|███████████████████▎                       | 44/98 [00:22<00:26,  2.00it/s]


 46%

  3%|█▎                                          | 3/98 [00:01<00:50,  1.90it/s]


  4%|█▊                                          | 4/98 [00:02<00:48,  1.94it/s]


  5%|██▏                                         | 5/98 [00:02<00:47,  1.97it/s]


  6%|██▋                                         | 6/98 [00:03<00:46,  1.99it/s]


  7%|███▏                                        | 7/98 [00:03<00:45,  1.99it/s]


  8%|███▌                                        | 8/98 [00:04<00:44,  2.00it/s]


  9%|████                                        | 9/98 [00:04<00:44,  2.00it/s]


 10%|████▍                                      | 10/98 [00:05<00:43,  2.01it/s]


 11%|████▊                                      | 11/98 [00:05<00:43,  2.01it/s]


 12%|█████▎                                     | 12/98 [00:06<00:42,  2.01it/s]


 13%|█████▋                                     | 13/98 [00:06<00:42,  2.01it/s]


 14%|██████▏                                    | 14/98 [00:07<00:41,  2.01it/s]


 15%

You can then view your inference in napari. It might be useful to you, especially if you are trying many inferences with modified parameters, to evalute the performance of each and track estimated % of correct and incorrect inference points in an Excel spreadsheet for example.

In [4]:
import napari
import numpy as np
import torchio as tio


image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_sp_FEG221129_366/FEG221129_366.nii'
image = tio.ScalarImage(image_path)

heatmap_path = '/home/jake/projects/dhr/output/FEG221129_366.logs_cystisoma_corneas_lightning_logs_version_14_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction.nii'
#Make sure to use resampled space csv here.
y_hat_path = '/home/jake/projects/dhr/output/FEG221129_366.logs_cystisoma_corneas_lightning_logs_version_14_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction_peak_min_val_0_5_method_center_of_mass.resampled_space_peaks.csv'
heatmap = tio.ScalarImage(heatmap_path)


viewer = napari.view_image(heatmap.numpy(), name='heatmap', contrast_limits=[0, 1])
#viewer.add_image(image.numpy(), name='image')

# load the coordinates to analyse
def load_coordinates(path, flip_axes=False, mct_path=None):
    locations = np.loadtxt(
        path,
        delimiter=',',
        ndmin=2,
        dtype=np.float64
    )
    if not flip_axes: 
        return locations.tolist()

    if not mct_path:
        raise Exception('You must specify `mct_path` if you need to flip_axes')

    mct = tio.ScalarImage(mct_path)
    locations[:,0] = mct.shape[1] - locations[:,0]
    locations[:,1] = mct.shape[2] - locations[:,1]

    return locations.tolist()


y_hat = load_coordinates(y_hat_path)


viewer.add_points(y_hat)


<Points layer 'Points' at 0x7fb85c398370>

## Modifying inference

In software such as MatLab you can modify the inference made from your model by adding and deleting points. This will be useful in retraining your model later.

##  (Re)training 

In [5]:
# We trained this model on a single scan, ran inference on it and then manually checked the labels. Now we are adding that labelled data to the dataset and retraining the model
# python check_data.py configs/cystisoma_corneas.yaml
# after updating the config to train for another 100,000 steps
# and changing the sigma from 2.5 to 2 (to suit the new data)
!python main.py train configs/cystisoma_corneas.yaml -w ./logs/cystisoma_corneas/lightning_logs/version_13/checkpoints/epoch=2809-step=699520.ckpt -ne 10

TorchIO version: 0.18.88
("Initiating model using the following config: {'seed': 44, 'lr': "
 "0.0424937961824346, 'num_steps': 800000, 'weight_decay': "
 "1.52709371682483e-06, 'momentum': 0.510999549987835, 'nesterov': False, "
 "'act': 'ReLU', 'output_activation': 'None', 'softargmax': False, "
 "'num_encoding_blocks': 5, 'out_channels_first_layer': 32, "
 "'double_channels_with_depth': True, 'pooling_type': 'max', "
 "'upsampling_type': 'conv', 'dropout': 0.0175756794552526, "
 "'loss_in_sigma_space': False, 'debug_plots': False, 'mse_with_f1': False, "
 "'visualise_model': False, 'learn_sigma': False, 'optimiser': 'SGD', "
 "'pin_memory': False, 'train_images_dir': "
 "'./dataset/cystisoma_corneas/whole/train_images_10/', 'train_labels_dir': "
 "'./dataset/cystisoma_corneas/whole/train_labels_10/', 'test_images_dir': "
 "'./dataset/cystisoma_corneas/whole/train_images_10/', 'test_labels_dir': "
 "'./dataset/cystisoma_corneas/whole/train_labels_10/', 'image_suffix': "
 "'image', 'l

# Cystisoma rhabdoms

## Checking

Check hyperparameters for heatmap. Modify these need be.

In [ ]:
!python check_data.py ./configs/cystisoma_rhabdoms.yaml

## (Re)training

After you finsihing checking the model you can train the model to rhabdoms and extend number of steps. 

In [2]:
#If you want to see all arguments 
!python main.py train -h

usage: main.py [-h] [--volume_path VOLUME_PATH]
               [--sql_storage_url SQL_STORAGE_URL]
               [--starting_weights_path STARTING_WEIGHTS_PATH]
               [--model_path MODEL_PATH] [--profile] [--num_steps NUM_STEPS]
               [--check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH]
               [--resample_ratio_path RESAMPLE_RATIO_PATH]
               [--bbox_path BBOX_PATH] [--n_x_dirs N_X_DIRS]
               [--n_y_dirs N_Y_DIRS] [--n_z_dirs N_Z_DIRS]
               [--resample_ratio RESAMPLE_RATIO] [--already_resampled]
               [--training_data_histogram] [--infer_with_max_filter_fun]
               [--average_threshold AVERAGE_THRESHOLD]
               [--inference_peak_min_val INFERENCE_PEAK_MIN_VAL]
               {train,tune,infer,locate_peaks} config_path

Train, hyperparameter tune or infer with a deep heatmap regression model

positional arguments:
  {train,tune,infer,locate_peaks}
                        Mode of operation
  config_path      

You can try training multiple models with different starting weights if necessary.

In [ ]:
# !python main.py train configs/cystisoma_rhabdoms.yaml -w /home/jake/projects/dhr/logs/paraphronima_rhabdoms/lightning_logs/version_23/checkpoints/epoch=99-step=537600-v1.ckpt -ne 10
!python main.py train configs/cystisoma_rhabdoms.yaml -w ./logs/paraphronima_rhabdoms_from_pretrained/lightning_logs/version_1/checkpoints/epoch=149-step=900000.ckpt #version_19
!python main.py train ./configs/cystisoma_rhabdoms.yaml -w ./logs/fiddlercrab_rhabdoms_from_pretrained/lightning_logs/version_1/checkpoints/epoch=123-step=865024.ckpt #version_20
# !python main.py train ./configs/cystisoma_rhabdoms_copy2.yaml -w ./logs/cystisoma_rhabdoms/lightning_logs/version_13/checkpoints/epoch=62679-step=4971328.ckpt -ne 10 #version_21

TorchIO version: 0.18.88
("Initiating model using the following config: {'seed': 44, 'lr': "
 "0.0424937961824346, 'num_steps': 1200000, 'weight_decay': "
 "1.52709371682483e-06, 'momentum': 0.510999549987835, 'nesterov': False, "
 "'act': 'ReLU', 'output_activation': 'None', 'softargmax': False, "
 "'num_encoding_blocks': 5, 'out_channels_first_layer': 32, "
 "'double_channels_with_depth': True, 'pooling_type': 'max', "
 "'upsampling_type': 'conv', 'dropout': 0.0175756794552526, "
 "'loss_in_sigma_space': False, 'debug_plots': False, 'mse_with_f1': False, "
 "'visualise_model': False, 'learn_sigma': False, 'optimiser': 'SGD', "
 "'pin_memory': False, 'train_images_dir': "
 "'./dataset/cystisoma_rhabdoms/whole/train_images_10/', 'train_labels_dir': "
 "'./dataset/cystisoma_rhabdoms/whole/train_labels_10/', 'test_images_dir': "
 "'./dataset/cystisoma_rhabdoms/whole/train_images_10/', 'test_labels_dir': "
 "'./dataset/cystisoma_rhabdoms/whole/train_labels_10/', 'image_suffix': "
 "'image

Epoch 150:  72%|▋| 92/128 [00:45<00:17,  2.02it/s, loss=9.99e-05, v_num=19, trai
Epoch 150:  73%|▋| 93/128 [00:45<00:17,  2.04it/s, loss=9.99e-05, v_num=19, trai
Epoch 150:  73%|▋| 94/128 [00:45<00:16,  2.06it/s, loss=9.99e-05, v_num=19, trai
Epoch 150:  74%|▋| 95/128 [00:45<00:15,  2.08it/s, loss=9.99e-05, v_num=19, trai
Epoch 150:  75%|▊| 96/128 [00:45<00:15,  2.10it/s, loss=9.99e-05, v_num=19, trai
Epoch 150:  76%|▊| 97/128 [00:45<00:14,  2.11it/s, loss=9.99e-05, v_num=19, trai
Epoch 150:  77%|▊| 98/128 [00:45<00:14,  2.13it/s, loss=9.99e-05, v_num=19, trai
Epoch 150:  77%|▊| 99/128 [00:46<00:13,  2.15it/s, loss=9.99e-05, v_num=19, trai
Epoch 150:  78%|▊| 100/128 [00:46<00:12,  2.17it/s, loss=9.99e-05, v_num=19, tra
Epoch 150:  79%|▊| 101/128 [00:46<00:12,  2.19it/s, loss=9.99e-05, v_num=19, tra
Epoch 150:  80%|▊| 102/128 [00:46<00:11,  2.20it/s, loss=9.99e-05, v_num=19, tra
Epoch 150:  80%|▊| 103/128 [00:46<00:11,  2.22it/s, loss=9.99e-05, v_num=19, tra
Epoch 150:  81%|▊| 104/128 [

Epoch 151:  95%|▉| 122/128 [01:04<00:03,  1.89it/s, loss=0.000451, v_num=19, tra
Epoch 151:  96%|▉| 123/128 [01:04<00:02,  1.90it/s, loss=0.000451, v_num=19, tra
Epoch 151:  97%|▉| 124/128 [01:04<00:02,  1.92it/s, loss=0.000451, v_num=19, tra
Epoch 151:  98%|▉| 125/128 [01:04<00:01,  1.93it/s, loss=0.000451, v_num=19, tra
Epoch 151:  98%|▉| 126/128 [01:04<00:01,  1.94it/s, loss=0.000451, v_num=19, tra
Epoch 151:  99%|▉| 127/128 [01:04<00:00,  1.95it/s, loss=0.000451, v_num=19, tra
Epoch 151: 100%|█| 128/128 [01:05<00:00,  1.97it/s, loss=0.000451, v_num=19, tra
Epoch 152:  50%|▌| 64/128 [00:24<00:24,  2.61it/s, loss=0.000666, v_num=19, trai
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                           | 0/64 [00:00<?, ?it/s]
Epoch 152:  51%|▌| 65/128 [00:41<00:40,  1.56it/s, loss=0.000666, v_num=19, trai
Epoch 152:  52%|▌| 66/128 [00:41<00:39,  1.58it/s, loss=0.000666, v_num=19, trai
Epoch 152:  52%|▌| 67/128 [00:41<00:38,  1.60it/s, loss=0.000666, v_num=19, tr

Epoch 153:  66%|▋| 84/128 [01:03<00:33,  1.33it/s, loss=0.00055, v_num=19, train
Epoch 153:  66%|▋| 85/128 [01:03<00:31,  1.35it/s, loss=0.00055, v_num=19, train
Epoch 153:  67%|▋| 86/128 [01:03<00:30,  1.36it/s, loss=0.00055, v_num=19, train
Epoch 153:  68%|▋| 87/128 [01:03<00:29,  1.37it/s, loss=0.00055, v_num=19, train
Epoch 153:  69%|▋| 88/128 [01:03<00:28,  1.39it/s, loss=0.00055, v_num=19, train
Epoch 153:  70%|▋| 89/128 [01:03<00:27,  1.40it/s, loss=0.00055, v_num=19, train
Epoch 153:  70%|▋| 90/128 [01:03<00:26,  1.42it/s, loss=0.00055, v_num=19, train
Epoch 153:  71%|▋| 91/128 [01:03<00:25,  1.43it/s, loss=0.00055, v_num=19, train
Epoch 153:  72%|▋| 92/128 [01:03<00:24,  1.44it/s, loss=0.00055, v_num=19, train
Epoch 153:  73%|▋| 93/128 [01:03<00:23,  1.46it/s, loss=0.00055, v_num=19, train
Epoch 153:  73%|▋| 94/128 [01:03<00:23,  1.47it/s, loss=0.00055, v_num=19, train
Epoch 153:  74%|▋| 95/128 [01:03<00:22,  1.49it/s, loss=0.00055, v_num=19, train
Epoch 153:  75%|▊| 96/128 [0

Epoch 154:  89%|▉| 114/128 [01:04<00:07,  1.75it/s, loss=0.000452, v_num=19, tra
Epoch 154:  90%|▉| 115/128 [01:05<00:07,  1.77it/s, loss=0.000452, v_num=19, tra
Epoch 154:  91%|▉| 116/128 [01:05<00:06,  1.78it/s, loss=0.000452, v_num=19, tra
Epoch 154:  91%|▉| 117/128 [01:05<00:06,  1.79it/s, loss=0.000452, v_num=19, tra
Epoch 154:  92%|▉| 118/128 [01:05<00:05,  1.81it/s, loss=0.000452, v_num=19, tra
Epoch 154:  93%|▉| 119/128 [01:05<00:04,  1.82it/s, loss=0.000452, v_num=19, tra
Epoch 154:  94%|▉| 120/128 [01:05<00:04,  1.83it/s, loss=0.000452, v_num=19, tra
Epoch 154:  95%|▉| 121/128 [01:05<00:03,  1.85it/s, loss=0.000452, v_num=19, tra
Epoch 154:  95%|▉| 122/128 [01:05<00:03,  1.86it/s, loss=0.000452, v_num=19, tra
Epoch 154:  96%|▉| 123/128 [01:05<00:02,  1.87it/s, loss=0.000452, v_num=19, tra
Epoch 154:  97%|▉| 124/128 [01:05<00:02,  1.89it/s, loss=0.000452, v_num=19, tra
Epoch 154:  98%|▉| 125/128 [01:05<00:01,  1.90it/s, loss=0.000452, v_num=19, tra
Epoch 154:  98%|▉| 126/128 [

...

Epoch 262:  97%|▉| 124/128 [01:01<00:01,  2.00it/s, loss=0.000135, v_num=19, tra
Epoch 262:  98%|▉| 125/128 [01:02<00:01,  2.01it/s, loss=0.000135, v_num=19, tra
Epoch 262:  98%|▉| 126/128 [01:02<00:00,  2.03it/s, loss=0.000135, v_num=19, tra
Epoch 262:  99%|▉| 127/128 [01:02<00:00,  2.04it/s, loss=0.000135, v_num=19, tra
Epoch 262: 100%|█| 128/128 [01:02<00:00,  2.05it/s, loss=0.000135, v_num=19, tra
Epoch 263:  50%|▌| 64/128 [00:40<00:40,  1.57it/s, loss=0.000174, v_num=19, trai
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                           | 0/64 [00:00<?, ?it/s]
Epoch 263:  51%|▌| 65/128 [00:57<00:55,  1.13it/s, loss=0.000174, v_num=19, trai
Epoch 263:  52%|▌| 66/128 [00:57<00:54,  1.14it/s, loss=0.000174, v_num=19, trai
Epoch 263:  52%|▌| 67/128 [00:57<00:52,  1.16it/s, loss=0.000174, v_num=19, trai
Epoch 263:  53%|▌| 68/128 [00:57<00:51,  1.17it/s, loss=0.000174, v_num=19, trai
Epoch 263:  54%|▌| 69/128 [00:58<00:49,  1.19it/s, loss=0.000174, v_num=19, tr

Epoch 264:  67%|▋| 86/128 [00:47<00:22,  1.83it/s, loss=0.000174, v_num=19, trai
Epoch 264:  68%|▋| 87/128 [00:47<00:22,  1.84it/s, loss=0.000174, v_num=19, trai
Epoch 264:  69%|▋| 88/128 [00:47<00:21,  1.86it/s, loss=0.000174, v_num=19, trai
Epoch 264:  70%|▋| 89/128 [00:47<00:20,  1.88it/s, loss=0.000174, v_num=19, trai
Epoch 264:  70%|▋| 90/128 [00:47<00:20,  1.90it/s, loss=0.000174, v_num=19, trai
Epoch 264:  71%|▋| 91/128 [00:47<00:19,  1.92it/s, loss=0.000174, v_num=19, trai
Epoch 264:  72%|▋| 92/128 [00:47<00:18,  1.93it/s, loss=0.000174, v_num=19, trai
Epoch 264:  73%|▋| 93/128 [00:47<00:17,  1.95it/s, loss=0.000174, v_num=19, trai
Epoch 264:  73%|▋| 94/128 [00:47<00:17,  1.97it/s, loss=0.000174, v_num=19, trai
Epoch 264:  74%|▋| 95/128 [00:47<00:16,  1.99it/s, loss=0.000174, v_num=19, trai
Epoch 264:  75%|▊| 96/128 [00:47<00:15,  2.00it/s, loss=0.000174, v_num=19, trai
Epoch 264:  76%|▊| 97/128 [00:47<00:15,  2.02it/s, loss=0.000174, v_num=19, trai
Epoch 264:  77%|▊| 98/128 [0

Epoch 265:  91%|▉| 116/128 [01:01<00:06,  1.87it/s, loss=0.000213, v_num=19, tra
Epoch 265:  91%|▉| 117/128 [01:01<00:05,  1.89it/s, loss=0.000213, v_num=19, tra
Epoch 265:  92%|▉| 118/128 [01:02<00:05,  1.90it/s, loss=0.000213, v_num=19, tra
Epoch 265:  93%|▉| 119/128 [01:02<00:04,  1.92it/s, loss=0.000213, v_num=19, tra
Epoch 265:  94%|▉| 120/128 [01:02<00:04,  1.93it/s, loss=0.000213, v_num=19, tra
Epoch 265:  95%|▉| 121/128 [01:02<00:03,  1.94it/s, loss=0.000213, v_num=19, tra
Epoch 265:  95%|▉| 122/128 [01:02<00:03,  1.96it/s, loss=0.000213, v_num=19, tra
Epoch 265:  96%|▉| 123/128 [01:02<00:02,  1.97it/s, loss=0.000213, v_num=19, tra
Epoch 265:  97%|▉| 124/128 [01:02<00:02,  1.98it/s, loss=0.000213, v_num=19, tra
Epoch 265:  98%|▉| 125/128 [01:02<00:01,  2.00it/s, loss=0.000213, v_num=19, tra
Epoch 265:  98%|▉| 126/128 [01:02<00:00,  2.01it/s, loss=0.000213, v_num=19, tra
Epoch 265:  99%|▉| 127/128 [01:02<00:00,  2.02it/s, loss=0.000213, v_num=19, tra
Epoch 265: 100%|█| 128/128 [

Epoch 267:  61%|▌| 78/128 [01:08<00:44,  1.14it/s, loss=0.000126, v_num=19, trai
Epoch 267:  62%|▌| 79/128 [01:08<00:42,  1.15it/s, loss=0.000126, v_num=19, trai
Epoch 267:  62%|▋| 80/128 [01:08<00:41,  1.16it/s, loss=0.000126, v_num=19, trai
Epoch 267:  63%|▋| 81/128 [01:08<00:39,  1.18it/s, loss=0.000126, v_num=19, trai
Epoch 267:  64%|▋| 82/128 [01:08<00:38,  1.19it/s, loss=0.000126, v_num=19, trai
Epoch 267:  65%|▋| 83/128 [01:09<00:37,  1.20it/s, loss=0.000126, v_num=19, trai
Epoch 267:  66%|▋| 84/128 [01:09<00:36,  1.22it/s, loss=0.000126, v_num=19, trai
Epoch 267:  66%|▋| 85/128 [01:09<00:35,  1.23it/s, loss=0.000126, v_num=19, trai
Epoch 267:  67%|▋| 86/128 [01:09<00:33,  1.24it/s, loss=0.000126, v_num=19, trai
Epoch 267:  68%|▋| 87/128 [01:09<00:32,  1.25it/s, loss=0.000126, v_num=19, trai
Epoch 267:  69%|▋| 88/128 [01:09<00:31,  1.27it/s, loss=0.000126, v_num=19, trai
Epoch 267:  70%|▋| 89/128 [01:09<00:30,  1.28it/s, loss=0.000126, v_num=19, trai
Epoch 267:  70%|▋| 90/128 [0

Epoch 268:  84%|▊| 108/128 [01:04<00:11,  1.67it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  85%|▊| 109/128 [01:04<00:11,  1.68it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  86%|▊| 110/128 [01:04<00:10,  1.69it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  87%|▊| 111/128 [01:05<00:09,  1.71it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  88%|▉| 112/128 [01:05<00:09,  1.72it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  88%|▉| 113/128 [01:05<00:08,  1.73it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  89%|▉| 114/128 [01:05<00:08,  1.75it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  90%|▉| 115/128 [01:05<00:07,  1.76it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  91%|▉| 116/128 [01:05<00:06,  1.77it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  91%|▉| 117/128 [01:05<00:06,  1.79it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  92%|▉| 118/128 [01:05<00:05,  1.80it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  93%|▉| 119/128 [01:05<00:04,  1.81it/s, loss=8.36e-05, v_num=19, tra
Epoch 268:  94%|▉| 120/128 [

## Inference

First open your heatmap in napari. Use the point selector to select some neighbouring rhabdoms and create a csv file of these selected points. You will use this to make your resampling ratio.

In [ ]:
# load in the nifti file with torchio
# 2nd scan
image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii'

image = tio.ScalarImage(image_path)


In [ ]:
image

In [ ]:
# view file with napari
import napari


viewer = napari.view_image(image.numpy(), name='image')


In napari, label a few of the features (that are next to each other) to estimate the resample ratio, then save it as a csv file and load it in the next step


In [2]:
import pandas as pd

#Read in your csv and assign it to the variable RR
# 2nd scan
RR = pd.read_csv('/home/jake/projects/dhr/Cystisoma_Hawaii_FEG20190213_rhabdoms_estimating_resampling_ratio.csv')

RR

,index,axis-0,axis-1,axis-2,axis-3
0,0.0,0.0,121.0,935.369470,943.075413
1,1.0,0.0,121.0,958.530798,938.844017
2,2.0,0.0,121.0,974.120153,958.442064
3,3.0,0.0,121.0,950.958825,966.682151
4,4.0,0.0,121.0,919.780115,926.817943
5,5.0,0.0,121.0,905.972400,935.726146
6,6.0,0.0,121.0,914.212488,957.105833
7,7.0,0.0,121.0,904.835604,908.432323
8,8.0,0.0,121.0,887.687313,916.672411
9,9.0,0.0,121.0,938.909480,921.571923


In [3]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-1', 'axis-2', 'axis-3']].values
dist_matrix = distance_matrix(coordinates, coordinates) #Create a distance matrix between points

In [4]:
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the dataframe
RR['nearest_distance'] = nearest_distances
RR

,index,axis-0,axis-1,axis-2,axis-3,nearest_distance
0,0.0,0.0,121.0,935.369470,943.075413,21.792930
1,1.0,0.0,121.0,958.530798,938.844017,22.846789
2,2.0,0.0,121.0,974.120153,958.442064,24.583453
3,3.0,0.0,121.0,950.958825,966.682151,24.583453
4,4.0,0.0,121.0,919.780115,926.817943,16.431953
5,5.0,0.0,121.0,905.972400,935.726146,16.431953
6,6.0,0.0,121.0,914.212488,957.105833,22.912662
7,7.0,0.0,121.0,904.835604,908.432323,19.025323
8,8.0,0.0,121.0,887.687313,916.672411,19.025323
9,9.0,0.0,121.0,938.909480,921.571923,19.835659


In [5]:
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance
resample_ratio = mean_nearest_distance/10
resample_ratio #Print your RR; use this to make your inference 

2.152958176307709

Now you can make your inference. Make sure you have the correct yaml, scan and model path, and have inserted your new resampling ratio. This will produce 3 files: 2 csv files (one resampled and the other not) with the predicted points, and a nifty file with your heatmap.

In [ ]:
!python main.py infer configs/cystisoma_rhabdoms.yaml -m ./logs/cystisoma_rhabdoms/lightning_logs/version_17/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5

/home/jake/projects/dhr/main.py:324: UserWarning: Volume path (`--volume_path`) was not specified. Running inference on test patches for debugging.
  warn(
TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
Tr

In [1]:
!python main.py -h

usage: main.py [-h] [--volume_path VOLUME_PATH]
               [--sql_storage_url SQL_STORAGE_URL]
               [--starting_weights_path STARTING_WEIGHTS_PATH]
               [--model_path MODEL_PATH] [--profile] [--num_steps NUM_STEPS]
               [--check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH]
               [--resample_ratio_path RESAMPLE_RATIO_PATH]
               [--bbox_path BBOX_PATH] [--n_x_dirs N_X_DIRS]
               [--n_y_dirs N_Y_DIRS] [--n_z_dirs N_Z_DIRS]
               [--resample_ratio RESAMPLE_RATIO] [--already_resampled]
               [--training_data_histogram] [--infer_with_max_filter_fun]
               [--average_threshold AVERAGE_THRESHOLD]
               [--inference_peak_min_val INFERENCE_PEAK_MIN_VAL]
               {train,tune,infer,locate_peaks} config_path

Train, hyperparameter tune or infer with a deep heatmap regression model

positional arguments:
  {train,tune,infer,locate_peaks}
                        Mode of operation
  config_path      

In [9]:
!python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_17/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.2 -at 0.2
!python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_17/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.3 -at 0.3
!python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_17/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.4 -at 0.4
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_18/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.5 -at 0.5

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_

[[ 272.46881809 1095.61649417 1648.20909275]
 [ 321.11574309 1372.89674499 1849.10672048]
 [ 319.35546282  989.64310838 1747.12556007]
 [ 325.09668462 1369.28140013 1840.77924074]
 [ 339.55226095  987.7156982  1742.11224318]
 [ 339.09091277 1020.2330558  1718.32974447]
 [ 351.21924383  704.30438474 1643.71180234]
 [ 355.23809909  708.32324001 1638.40117217]
 [ 362.61966998  749.22944536 1662.69884302]
 [ 449.96825885  585.60462396 1625.48342311]
 [ 524.47365998  309.50404813 1079.61066368]
 [ 540.39250225  501.63925508 1754.66091369]
 [ 557.05748864  884.71592097 1952.24251848]
 [ 591.10662818  711.79189484 1110.44798382]
 [ 588.83406122  731.08308358 1879.68627064]
 [ 601.03415755  768.60606894 1772.60223183]
 [ 610.36364298  766.99135031 1767.0404232 ]
 [ 639.2629662  1453.57799334 1941.63705069]
 [ 709.39971909  377.84415994 1802.02599357]
 [ 732.29284103 1289.04782543  411.9326644 ]
 [ 753.73108518  512.7954929  1913.00120138]
 [ 751.38240353  512.40404596 1935.5094005 ]
 [ 864.197

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_

In [6]:
!python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_17/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.2

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_

  4%|█▋                                         | 5/126 [00:02<01:02,  1.93it/s]


  5%|██                                         | 6/126 [00:03<01:01,  1.95it/s]


  6%|██▍                                        | 7/126 [00:03<01:00,  1.97it/s]


  6%|██▋                                        | 8/126 [00:04<00:59,  1.98it/s]


  7%|███                                        | 9/126 [00:04<00:58,  1.99it/s]


  8%|███▎                                      | 10/126 [00:05<00:57,  2.00it/s]


  9%|███▋                                      | 11/126 [00:05<00:57,  2.00it/s]


 10%|████                                      | 12/126 [00:06<00:57,  1.99it/s]


 10%|████▎                                     | 13/126 [00:06<00:56,  2.00it/s]


 11%|████▋                                     | 14/126 [00:07<00:56,  1.97it/s]


 12%|█████                                     | 15/126 [00:07<00:56,  1.98it/s]


 13%|█████▎                                    | 16/126 [00:08<00:55,  1.98it/s]


 13%

 40%|█████████████████                         | 51/126 [00:25<00:37,  1.99it/s]


 41%|█████████████████▎                        | 52/126 [00:26<00:37,  1.99it/s]


 42%|█████████████████▋                        | 53/126 [00:26<00:36,  1.99it/s]


 43%|██████████████████                        | 54/126 [00:27<00:36,  1.99it/s]


 44%|██████████████████▎                       | 55/126 [00:27<00:35,  2.00it/s]


 44%|██████████████████▋                       | 56/126 [00:28<00:35,  2.00it/s]


 45%|███████████████████                       | 57/126 [00:28<00:35,  1.96it/s]


 46%|███████████████████▎                      | 58/126 [00:29<00:35,  1.94it/s]


 47%|███████████████████▋                      | 59/126 [00:29<00:34,  1.92it/s]


 48%|████████████████████                      | 60/126 [00:30<00:33,  1.95it/s]


 48%|████████████████████▎                     | 61/126 [00:30<00:33,  1.97it/s]


 49%|████████████████████▋                     | 62/126 [00:31<00:32,  1.98it/s]


 50%

  5%|██                                         | 6/126 [00:03<01:00,  1.97it/s]


  6%|██▍                                        | 7/126 [00:03<00:59,  1.99it/s]


  6%|██▋                                        | 8/126 [00:04<00:59,  1.99it/s]


  7%|███                                        | 9/126 [00:04<00:59,  1.97it/s]


  8%|███▎                                      | 10/126 [00:05<00:59,  1.96it/s]


  9%|███▋                                      | 11/126 [00:05<00:59,  1.92it/s]


 10%|████                                      | 12/126 [00:06<00:59,  1.91it/s]


 10%|████▎                                     | 13/126 [00:06<00:59,  1.90it/s]


 11%|████▋                                     | 14/126 [00:07<00:58,  1.91it/s]


 12%|█████                                     | 15/126 [00:07<00:57,  1.93it/s]


 13%|█████▎                                    | 16/126 [00:08<00:56,  1.95it/s]


 13%|█████▋                                    | 17/126 [00:08<00:55,  1.97it/s]


 14%

 41%|█████████████████▎                        | 52/126 [00:26<00:37,  1.99it/s]


 42%|█████████████████▋                        | 53/126 [00:26<00:37,  1.96it/s]


 43%|██████████████████                        | 54/126 [00:27<00:36,  1.96it/s]


 44%|██████████████████▎                       | 55/126 [00:27<00:36,  1.96it/s]


 44%|██████████████████▋                       | 56/126 [00:28<00:35,  1.96it/s]


 45%|███████████████████                       | 57/126 [00:28<00:35,  1.97it/s]


 46%|███████████████████▎                      | 58/126 [00:29<00:34,  1.98it/s]


 47%|███████████████████▋                      | 59/126 [00:29<00:33,  1.99it/s]


 48%|████████████████████                      | 60/126 [00:30<00:33,  2.00it/s]


 48%|████████████████████▎                     | 61/126 [00:30<00:32,  2.00it/s]


 49%|████████████████████▋                     | 62/126 [00:31<00:31,  2.00it/s]


 50%|█████████████████████                     | 63/126 [00:31<00:31,  1.98it/s]


 51%

...

 48%|████████████████████▎                     | 61/126 [00:30<00:32,  2.01it/s]


 49%|████████████████████▋                     | 62/126 [00:30<00:31,  2.01it/s]


 50%|█████████████████████                     | 63/126 [00:31<00:31,  2.01it/s]


 51%|█████████████████████▎                    | 64/126 [00:31<00:30,  2.01it/s]


 52%|█████████████████████▋                    | 65/126 [00:32<00:30,  2.00it/s]


 52%|██████████████████████                    | 66/126 [00:32<00:29,  2.01it/s]


 53%|██████████████████████▎                   | 67/126 [00:33<00:29,  2.01it/s]


 54%|██████████████████████▋                   | 68/126 [00:33<00:28,  2.01it/s]


 55%|███████████████████████                   | 69/126 [00:34<00:28,  2.01it/s]


 56%|███████████████████████▎                  | 70/126 [00:34<00:27,  2.01it/s]


 56%|███████████████████████▋                  | 71/126 [00:35<00:27,  2.01it/s]


 57%|████████████████████████                  | 72/126 [00:35<00:26,  2.01it/s]


 58%

 83%|█████████████████████████████████▊       | 104/126 [00:52<00:10,  2.01it/s]


 83%|██████████████████████████████████▏      | 105/126 [00:52<00:10,  2.01it/s]


 84%|██████████████████████████████████▍      | 106/126 [00:53<00:09,  2.01it/s]


 85%|██████████████████████████████████▊      | 107/126 [00:53<00:09,  2.00it/s]


 86%|███████████████████████████████████▏     | 108/126 [00:54<00:08,  2.00it/s]


 87%|███████████████████████████████████▍     | 109/126 [00:54<00:08,  1.98it/s]


 87%|███████████████████████████████████▊     | 110/126 [00:55<00:08,  1.99it/s]


 88%|████████████████████████████████████     | 111/126 [00:55<00:07,  1.99it/s]


 89%|████████████████████████████████████▍    | 112/126 [00:56<00:07,  1.99it/s]


 90%|████████████████████████████████████▊    | 113/126 [00:56<00:06,  2.00it/s]


 90%|█████████████████████████████████████    | 114/126 [00:57<00:06,  2.00it/s]


 91%|█████████████████████████████████████▍   | 115/126 [00:57<00:05,  2.00it/s]


 92%

 14%|██████                                    | 18/126 [00:09<00:53,  2.02it/s]


 15%|██████▎                                   | 19/126 [00:09<00:53,  2.02it/s]


 16%|██████▋                                   | 20/126 [00:10<00:52,  2.01it/s]


 17%|███████                                   | 21/126 [00:10<00:52,  2.01it/s]


 17%|███████▎                                  | 22/126 [00:11<00:51,  2.01it/s]


 18%|███████▋                                  | 23/126 [00:11<00:51,  2.01it/s]


 19%|████████                                  | 24/126 [00:12<00:50,  2.01it/s]


 20%|████████▎                                 | 25/126 [00:12<00:50,  2.01it/s]


 21%|████████▋                                 | 26/126 [00:13<00:49,  2.01it/s]


 21%|█████████                                 | 27/126 [00:13<00:49,  2.01it/s]


 22%|█████████▎                                | 28/126 [00:14<00:48,  2.01it/s]


 23%|█████████▋                                | 29/126 [00:14<00:48,  2.01it/s]


 24%

 48%|████████████████████▎                     | 61/126 [00:30<00:32,  2.01it/s]


 49%|████████████████████▋                     | 62/126 [00:30<00:31,  2.00it/s]


 50%|█████████████████████                     | 63/126 [00:31<00:31,  2.01it/s]


 51%|█████████████████████▎                    | 64/126 [00:31<00:30,  2.01it/s]


 52%|█████████████████████▋                    | 65/126 [00:32<00:30,  2.01it/s]


 52%|██████████████████████                    | 66/126 [00:32<00:29,  2.01it/s]


 53%|██████████████████████▎                   | 67/126 [00:33<00:29,  2.01it/s]


 54%|██████████████████████▋                   | 68/126 [00:33<00:28,  2.01it/s]


 55%|███████████████████████                   | 69/126 [00:34<00:28,  2.01it/s]


 56%|███████████████████████▎                  | 70/126 [00:34<00:27,  2.01it/s]


 56%|███████████████████████▋                  | 71/126 [00:35<00:27,  2.01it/s]


 57%|████████████████████████                  | 72/126 [00:35<00:26,  2.01it/s]


 58%

 85%|██████████████████████████████████▊      | 107/126 [00:53<00:09,  2.00it/s]


 86%|███████████████████████████████████▏     | 108/126 [00:53<00:08,  2.00it/s]


 87%|███████████████████████████████████▍     | 109/126 [00:54<00:08,  2.00it/s]


 87%|███████████████████████████████████▊     | 110/126 [00:54<00:07,  2.00it/s]


 88%|████████████████████████████████████     | 111/126 [00:55<00:07,  2.01it/s]


 89%|████████████████████████████████████▍    | 112/126 [00:55<00:06,  2.00it/s]


 90%|████████████████████████████████████▊    | 113/126 [00:56<00:06,  2.00it/s]


 90%|█████████████████████████████████████    | 114/126 [00:56<00:05,  2.01it/s]


 91%|█████████████████████████████████████▍   | 115/126 [00:57<00:05,  2.00it/s]


 92%|█████████████████████████████████████▋   | 116/126 [00:57<00:04,  2.00it/s]


 93%|██████████████████████████████████████   | 117/126 [00:58<00:04,  2.00it/s]


 94%|██████████████████████████████████████▍  | 118/126 [00:58<00:04,  2.00it/s]


 94%

You can then view your inference in napari.

In [6]:
import napari
import numpy as np
import torchio as tio


image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii'
image = tio.ScalarImage(image_path)

# heatmap_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction.nii'
heatmap_path = '/home/jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_18_checkpoints_last_x_1_y_1_z_1_average_threshold_0.2_prediction.nii'

#Make sure to use resampled space csv here.
# y_hat_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction_peak_min_val_0_5_method_center_of_mass.resampled_space_peaks.csv'
y_hat_path = '/home/jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_18_checkpoints_last_x_1_y_1_z_1_average_threshold_0.2_prediction_peak_min_val_0_2_method_center_of_mass.resampled_space_peaks.csv'

heatmap = tio.ScalarImage(heatmap_path)


viewer = napari.view_image(heatmap.numpy(), name='heatmap', contrast_limits=[0, 1])
# viewer.add _image(image.numpy(), name='image') #If you want to view your scan also (not necessary)

# load the coordinates to analyse
def load_coordinates(path, flip_axes=False, mct_path=None):
    locations = np.loadtxt(
        path,
        delimiter=',',
        ndmin=2,
        dtype=np.float64
    )
    if not flip_axes: 
        return locations.tolist()

    if not mct_path:
        raise Exception('You must specify `mct_path` if you need to flip_axes')

    mct = tio.ScalarImage(mct_path)
    locations[:,0] = mct.shape[1] - locations[:,0]
    locations[:,1] = mct.shape[2] - locations[:,1]

    return locations.tolist()


y_hat = load_coordinates(y_hat_path)


viewer.add_points(y_hat)

<Points layer 'Points' at 0x7f38c49e98b0>